In [2]:
# Import all the needed libraries
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import algos
import os
if not pt.started():
    pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [8]:
# Load the dataset
dataset = pt.datasets.get_dataset("trec-deep-learning-passages")

In [9]:
index_ref = pt.IndexRef.of(os.getcwd() + "./passage_index/data.properties")

In [10]:
index = pt.IndexFactory.of(index_ref)

14:57:16.921 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.


In [18]:
pipelineQE = pt.BatchRetrieve(index, wmodel="Bo1", controls={"qemodel" : "Bo1", "qe" : "on"})

In [19]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [20]:
print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [21]:
sdm = pt.rewrite.SequentialDependence()
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipelineSD = sdm >> dph

In [25]:
# pt.Experiment(
#   [bm25, pipelineQE, pipelineSD],
#   dataset.get_topics("test-2019"), 
#   dataset.get_qrels("test-2019"),
#   eval_metrics=["ndcg", "map", "recip_rank"], 
#   # perquery = "true",
#   names=["BM25", "Bo1", "DPH"],
#   baseline=0,
#   round=3,
# )

In [53]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25")
Bo1 = pt.rewrite.Bo1QueryExpansion(index)
KL = pt.rewrite.KLQueryExpansion(index)

In [54]:
bm25_pipe = BM25 >> pt.apply.query(printQuery)
Bo1_pipe = BM25 >> Bo1 >> BM25 >> pt.apply.query(printQuery)
KL_pipe = BM25 >> KL >> BM25 >> pt.apply.query(printQuery)

In [58]:
def printQuery(q):
    print(q['query'])

In [59]:
pt.Experiment(
    [
            BM25, 
            Bo1_pipe, 
            KL_pipe
    ],
    dataset.get_topics("test-2019").sort_values(by=['qid']).head(10),
    dataset.get_qrels("test-2019").sort_values(by=['qid']).head(10),
    eval_metrics=["map", "P_10", "P_20", "ndcg_cut_20"],
    names=["BM25", "+Bo1", "+KL"],
    baseline=0,
    filter_by_qrels=1,
    round=3,
    )

16:35:57.921 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8
applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000 applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000
applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000 applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 yea

applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000 applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000
applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000 applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000
applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.0000000

applypipeline:off who^1.065015991 robert^1.441999714 grai^1.465145958 mississippi^0.096572198 governor^0.094231346 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000 applypipeline:off who^1.065015991 robert^1.441999714 grai^1.465145958 mississippi^0.096572198 governor^0.094231346 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000
applypipeline:off who^1.065015991 robert^1.441999714 grai^1.465145958 mississippi^0.096572198 governor^0.094231346 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000 applypipeline:off who^1.065015991 robert^1.441999714 grai^1.465145958 mississippi^0.096572198 governor^0.094231346 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000
applypipeline:off who^1.065015991 robert^1.441999714 grai^1.465145958 mississippi^0.096572198 governor^0.094231346 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.0000000

C:\Users\Jbies\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Jbies\anaconda3\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,name,map,P_10,P_20,ndcg_cut_20,map +,map -,map p-value,P_10 +,P_10 -,P_10 p-value,P_20 +,P_20 -,P_20 p-value,ndcg_cut_20 +,ndcg_cut_20 -,ndcg_cut_20 p-value
0,BM25,0.040,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,+Bo1,0.034,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN
2,+KL,0.034,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN


In [ ]:
5015991 robert^1.441999714 grai^1.465145958 mississippi^0.096572198 governor^0.094231346 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000

In [88]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")


# Divergence
qe = (pt.BatchRetrieve(index_ref, wmodel="BM25") >> 
    pt.rewrite.QueryExpansion(index_ref) >> 
    pt.BatchRetrieve(index_ref, wmodel="BM25")
)



expers = pt.Experiment(
  [bm25, qe],
  dataset.get_topics("test-2019"), 
  dataset.get_qrels("test-2019"),
  eval_metrics=["ndcg", "map", "recip_rank"], 
  names=["BM25", "+Div"],
  filter_by_qrels=1,
  perquery="true",
  round=3,
)

10:57:27.819 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.
10:57:28.852 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.
10:57:29.957 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.
10:57:29.969 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8
10:57:39.134 [main] WARN org.terrier.querying.QueryExpansion - qe

In [91]:
expers[expers['qid']==str(1037798)]

,name,qid,measure,value
129,+Div,1037798,map,0.139
130,+Div,1037798,recip_rank,0.250
131,+Div,1037798,ndcg,0.440
27,BM25,1037798,map,0.110
28,BM25,1037798,recip_rank,0.333
29,BM25,1037798,ndcg,0.433


In [79]:
(bm25.search("who is robert gray")).iloc[0]

qid                       1
docid               8760867
docno               8760867
rank                      0
score             30.522236
query    who is robert gray
Name: 0, dtype: object

In [84]:
(qe.search("who is robert gray")).iloc[0]['query']

'applypipeline:off who^1.075470753 robert^1.409366018 grai^1.433798694 mississippi^0.112481464 governor^0.110010191 will^0.000000000 columbia^0.000000000 name^0.000000000 year^0.000000000 made^0.000000000'

In [82]:
algos.search_with_doc_id(int(8760867))

"Robert Gray. A surprise came on the Democratic side in the race for Mississippi Governor. Robert Gray, a retired firefighter and truck driver, is the democratic nominee who admitted to the Associated Press that he didn't even vote in Tuesday's election.\n"